In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as pe
from AVN import *
from scipy.stats import norm
import scipy.io as scp
import os

## Load data:

In [ ]:
with open('v0_d03_n50_every1_clusters/X95_Y95_inter.npy', 'rb') as f:
    X95 = np.load(f) ##count begin at 1 and not 0
    Y95 = np.load(f) ##count begin at 1 and not 0
    inter = np.load(f) ##count begin at 1 and not 0
    
#print(X95,Y95,inter) ##ok!

In [ ]:
dfI = pd.read_csv("/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas_23052024_2/new_all_run_files_for_etas/v0_d03_n50_every1_clusters/df_integral.csv")

dfI.head()

In [ ]:
dfM = pd.read_csv("/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas_23052024_2/new_all_run_files_for_etas/v0_d03_n50_every1_clusters/dfM.csv")
dfM['N'] = dfM['N'].astype(int)

dfM.index = dfM.I.values ##count begin at 1 and not 0

#dfM = dfM.drop(['I'], axis=1)

dfM.head()

In [ ]:
dfNM = pd.read_csv("/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas_23052024_2/new_all_run_files_for_etas/v0_d03_n50_every1_clusters/dfNM.csv")
dfNM['N'] = dfNM['N'].astype(int)

dfNM.index = dfNM.I.values ##count begin at 1 and not 0

#dfNM = dfNM.drop(['I'], axis=1)

dfNM.head()

In [ ]:
dfA = pd.read_csv("/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas/v0_d03_n50_every1_clusters/quick_stats_all_swarms.csv")
dfA['N'] = dfA['N'].astype(int)

dfA.index = dfA.I.values ##count begin at 1 and not 0

#dfA = dfA.drop(['I'], axis=1)
##Renew dataframe A:
#dfA.to_csv("/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas/v0_d03_n50_every1_clusters/quick_stats_all_swarms.csv")
##run only once

dfA#.head()

## Check histograms:

In [ ]:
nbins = 5

plt.figure(figsize=(15,10))
plt.subplot(2,3,1)
plt.hist(dfA.N, nbins)
plt.yticks(np.arange(0,40,5))
plt.tick_params("both", labelsize=15)
plt.xlabel("Total number of earthquakes $N$", fontsize=15)
plt.ylabel("Cumulative seismic swarms", fontsize=15)
plt.grid()

plt.subplot(2,3,2)
plt.hist(dfA.max_m, nbins, label=['all swarms', 'swarms-like with migration','swarms-like without migration'])
plt.yticks(np.arange(0,40,5))
plt.tick_params("both", labelsize=15)
plt.xlabel("Maximum magnitude $m_L$", fontsize=15)
plt.grid()

plt.subplot(2,3,3)
plt.hist(dfA.V2, nbins)
plt.xticks(np.arange(0,1+0.2,0.2))
plt.yticks(np.arange(0,40,5))
plt.tick_params("both", labelsize=15)
plt.xlabel("Volume (km³)", fontsize=15)
plt.grid()

plt.subplot(2,3,4)
plt.hist(dfA.plana, nbins)
plt.xticks(np.arange(0,1+0.2,0.2))
plt.yticks(np.arange(0,40,5))
plt.tick_params("both", labelsize=15)
plt.xlabel("Planarity", fontsize=15)
plt.ylabel("Cumulative seismic swarms", fontsize=15)
plt.grid()

plt.subplot(2,3,5)
plt.hist(dfA.duration, nbins)
plt.yticks(np.arange(0,40,5))
plt.tick_params("both", labelsize=15)
plt.xlabel("Duration (d)", fontsize=15)
plt.grid()

# plt.subplot(2,3,6)
# plt.hist(dfA.v, nbins)
# plt.xticks(np.arange(0,1+0.2,0.2))
# plt.yticks(np.arange(0,40,5))
# plt.tick_params("both", labelsize=15)
# plt.xlabel("Migration velocity (km/d)", fontsize=15)
# plt.grid()

plt.tight_layout()
plt.savefig("v0_d03_n50_every1_clusters/global_stats.png")
plt.show()

## Load EQ catalog:

In [ ]:
path_to_df0 = "/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas_23052024_2/new_all_run_files_for_etas/catalog_all/"

####Raw data:
df0 = pd.read_csv(path_to_df0+'raw_data_modified_version.csv')
###Structure : t, x, y, z, m, x_km, y_km

###Swarms obtained by after ETAS & seismicity rate & checking step:
df_swarms = pd.read_csv('v0_d03_n50_every1_clusters/new_df_final_swarms_of_v0_d03_n50_every1.csv')

##Rename swarms after sorting along t1:
df_swarms = df_swarms.sort_values("t1")
df_swarms.index = np.arange(1, len(df_swarms)+1)

df_swarms###Create a column of swarms number:
df0["swarms_num"] = np.zeros(len(df0)) -1

for i in range(0, len(df_swarms)):
    select_swarms = df0.index[(df0.t >= df_swarms.t1.iloc[i])&(df0.t <= df_swarms.t2.iloc[i])
                       & (df0.x >= df_swarms.x1.iloc[i])&(df0.x <= df_swarms.x2.iloc[i])
                       & (df0.y >= df_swarms.y1.iloc[i])&(df0.y <= df_swarms.y2.iloc[i])
                       & (df0.z >= df_swarms.z1.iloc[i])&(df0.z <= df_swarms.z2.iloc[i])]
    
    df0.swarms_num.iloc[select_swarms] = df_swarms.index[i]   
    
    
df0['swarms_num'] = df0['swarms_num'].astype(int)

list_swarms = np.unique(df0["swarms_num"].values)[1:]
print(list_swarms)

###Remove all rows with a magnitude equal to a nan value:
df0 = df0.dropna()

df0.index = np.arange(0, len(df0))

df0

In [ ]:
###Rescale on Norcia event:
xN_km = df0.x_km - N.x_km
yN_km = df0.y_km - N.y_km

df0["xN_km"] = xN_km
df0["yN_km"] = yN_km

###Define AVN MS:
A, V, N, df_AVN = AVN(df0)

###Define EQ m >= 5: ##associated to gold color
df0_m5 = df0[df0.m >= 5]


In [ ]:
def statistics(list_swarm) :

    all_tmax = []
    all_tcentroid = []
    all_skew = []
    all_kurt = []
    all_dm = []
    
    all_tm = [] ##eifference with all_tmax is all_tm is normalized by mean t


    for num_swarm in list_swarm:

        df_clust = 0

        ##Select data:
        df_clust = df0[df0.swarms_num == num_swarm]
        df_clust.index=np.arange(len(df_clust))

        ##Compute new t: réinitialiser le t0
        df_clust["new_t"] = df_clust.t - df_clust.t.iloc[0] 
        
        ##Sort magnitude:
        df_clust_m = df_clust.sort_values("m", ascending = False) ##du + gd au plus petit
        
        ##Get the time of the largest EQ inside the swarm:
        tmax = df_clust_m.new_t.iloc[0]
        all_tmax.append(tmax)
        tm = df_clust_m.new_t.iloc[0] / df_clust_m.new_t.mean()
        all_tm.append(tm)
        
        #######ROLAND AND MCGUIRE, 2009:
        ##Compute seismic moment m0(i):
        m0_i = 10**(1.5*df_clust.m.values + 9.1)
        m0_i = m0_i / np.sum(m0_i) ##np.sum(m0_i) car normalisé
        m_tot = np.sum(m0_i)
        
        ##Compute mean of new_t:
        tbar = np.sum((df_clust.new_t.values * m0_i)) / m_tot
        tcentroid = tbar
        all_tcentroid.append(tcentroid)
        
        ##Compute sigma:
        sigma = np.sqrt(np.sum(((df_clust.new_t.values - tbar)**2)* m0_i))
        
        ##Compute the skewness:
        skew = np.sum(((df_clust.new_t.values - tbar)**3)* m0_i) / (sigma**3)
        all_skew.append(skew)
        
        ##Compute the kurtosis:
        kurt = np.sum(((df_clust.new_t.values - tbar)**4)* m0_i) / (sigma**4)
        all_kurt.append(kurt)
        
        ##Compute difference in magnitude between the 2 largest EQ within the swarm:
        dm = df_clust_m.m.iloc[0] - df_clust_m.m.iloc[1]
        all_dm.append(dm)
    
    all_tmax = np.array(all_tmax)
    all_tcentroid = np.array(all_tcentroid)
    all_skew = np.array(all_skew)
    all_kurt = np.array(all_kurt)
    all_dm = np.array(all_dm)
    all_tm = np.array(all_tm)
    
    data = {"tmax":all_tmax,
            "tcentroid":all_tcentroid,
            "skew": all_skew,
            "kurt":all_kurt,
            "dm": all_dm,
            "tm": all_tm}
    
    df_final = pd.DataFrame(data)
    
    return df_final

In [ ]:
print(list_swarms)
df_swarms_characteritics = statistics(list_swarms)
df_swarms_characteritics

In [ ]:
df_final_swarms_ratio = pd.read_csv('/home/xianglo/Bureau/DATA_AMATRICE/DBSCAN_David/new_all_run_files_for_etas_23052024_2/new_all_run_files_for_etas/v0_d03_n50_every1_clusters/df_final_swarms_of_v0_d03_n50_every5_with_ratio.csv')

df_final_swarms_ratio = df_final_swarms_ratio.drop([12]) ##remove AF
df_final_swarms_ratio.index = np.arange(0, len(df_final_swarms_ratio))

df_final_swarms_ratio

In [ ]:
df_swarms_characteritics_ = df_swarms_characteritics.copy()
df_swarms_characteritics_ = pd.concat([df_swarms_characteritics_, df_final_swarms_ratio.iloc[:,9:]], axis=1)
df_swarms_characteritics_.index = np.arange(1, len(df_swarms_characteritics_)+1)

df_swarms_characteritics_#.Q3.quantile([0.25,0.5,0.75])

In [ ]:
msz=500

###From Mesimeri et al, 2019:
###Parabola of 95% confidence limits:
a = [1.03, 1.14]
b = [0.41, 1.31]
c = [-30.79, 43.60]

x = np.arange(-10, 20, 0.1)

p1 = a[0]*x**2 + b[0]*x + c[0] ##parabole en bas
p2 = a[1]*x**2 + b[1]*x + c[1] ##parabole en haut


select_cmap = "jet"
clim1, clim2 = 1, 6

plt.figure(figsize=(12,5))
for i in range(1,40):
    if dfM[dfM.I.values == i].empty == False:
        I = df_swarms_characteritics_[df_swarms_characteritics_.index == i]
#         print(i, I["skew"].values, I["kurt"].values, np.log(I["Q3"].values))
        plt.scatter(I["skew"].values, I["kurt"].values, s=msz, marker='s', c=np.log(I["Q3"].values), cmap=select_cmap, edgecolors='k', linewidths=1, alpha = 0.8)
        plt.clim([clim1,clim2])
#         plt.annotate(str(int(i)), (I["skew"].values, I["kurt"].values),
#                 color="white", alpha =1, size=10, 
#                 path_effects=[pe.withStroke(linewidth=3, foreground="k")])         
    else:
        I = df_swarms_characteritics_[df_swarms_characteritics_.index == i]
#         print(i, I["skew"].values, I["kurt"].values, np.log(I["Q3"].values))
        plt.scatter(I["skew"].values, I["kurt"].values, s=msz, c=np.log(I["Q3"].values), cmap=select_cmap, edgecolors='k', linewidths=1, alpha = 0.8)
        plt.clim([clim1,clim2])
#         plt.annotate(str(int(i)), (I["skew"], I["kurt"]),
#                 color="white", alpha =1, size=10, 
#                 path_effects=[pe.withStroke(linewidth=3, foreground="k")])
            
# plt.plot(x, p1, ls='--', c='k')
# plt.plot(x, p2, ls='--', c='k')

plt.axvline(x=3.5, ls = '--', c='m')
plt.axhline(y=22, ls = '--', c='m')
    
plt.xlabel("Skewness $S$", fontsize=20)
plt.ylabel("Kurtosis $K$", fontsize=20)
plt.xlim([-5, 5])
plt.ylim([-25, 50])
cb = plt.colorbar(pad=0.015)
cb.set_label(label='Ratio log($R$)', size=20)
cb.ax.tick_params(labelsize=15)
plt.tick_params(axis='both',labelsize=15)
plt.grid()
plt.savefig("v0_d03_n50_every1_clusters/skew_vs_kurt_zoom.png")
plt.show()